<a href="https://colab.research.google.com/github/Gus-oss/DeepLearning/blob/main/Perceptros_Lotes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Librerias importantes
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

#Coneccion con google drive
from google.colab import drive
from google.colab import files
#Importar la carpeta de drive
drive.mount('/content/drive') #coneccion con drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df = pd.read_excel("/content/drive/MyDrive/DeepLearning/archivos/EarlyRetirementPrediction.csv.xlsx")

In [8]:
sc = StandardScaler() # Standardisación
scaled = sc.fit_transform(df)
res = pd.DataFrame(scaled, columns=df.columns) #este es el df que utilizaremos

In [9]:
#utilizaremos el conjunto res, puesto ya esta estandarizado
X_columns = ['Gender', 'Desease', 'Education Level', 'Marital Status',
             'Monthly Income', 'Employee/Employer', 'Dependants',
             'Unemployment Rate', 'Stock Market', 'Credit Score',
             'Government Bonds Return']
X_raw = res[X_columns].values
y = df['Retire Before 65 Years Old'].values  # Tomamos y del conjnto original y no del estandarizado

In [11]:
#declaramos las funciones de la simoide, la funcion de costos y el gradiente
def sigmoide(z):
    z = np.clip(z, -500, 500)#Evita overflow
    return 1 / (1 + np.exp(-z))


def cross_entropy_loss(y, y_hat):
    epsilon = 1e-15 #para evitar el cero
    y_hat = np.clip(y_hat, epsilon, 1 - epsilon)

    loss = -np.sum(y * np.log(y_hat))
    return loss

def gradientes_cross_entropy(X, y, y_hat):
    dL_dz = - y * (1 - y_hat) # dL/dz para cada observación
    grad_w = X.T @ dL_dz #gradientes
    grad_b = np.sum(dL_dz)

    return grad_w, grad_b

In [12]:
#formamos una funcion para la propagacion hacia adelante
def forward_pass(X, w, b):
    z = X @ w + b
    y_hat = sigmoide(z)
    return z, y_hat

In [17]:
# Inicialización
n_features = X_raw.shape[1]
N = X_raw.shape[0]

np.random.seed(42)
w = np.random.randn(n_features) * 0.1
b = 0.0

In [14]:
# Realizamos el entrenamiento por lotes
batch_size = 100
eta = 0.01
n_batches = N // batch_size

print(f"\n{'='*60}")
print(f"ENTRENAMIENTO POR LOTES (batch_size = {batch_size})")
print(f"Número de lotes: {n_batches}")
print(f"{'='*60}")

for lote in range(n_batches):
    inicio = lote * batch_size
    fin = inicio + batch_size

    X_batch = X_raw[inicio:fin]
    y_batch = y[inicio:fin]

    z_batch, y_hat_batch = forward_pass(X_batch, w, b)
    L_batch = cross_entropy_loss(y_batch, y_hat_batch)

    grad_w, grad_b = gradientes_cross_entropy(X_batch, y_batch, y_hat_batch)

    w = w - eta * grad_w
    b = b - eta * grad_b

    print(f"Lote {lote+1:2d}/{n_batches} | Índices [{inicio:4d}:{fin:4d}] | Loss lote: {L_batch:.4f}")


ENTRENAMIENTO POR LOTES (batch_size = 100)
Número de lotes: 15
Lote  1/15 | Índices [   0: 100] | Loss lote: 30.9147
Lote  2/15 | Índices [ 100: 200] | Loss lote: 20.5003
Lote  3/15 | Índices [ 200: 300] | Loss lote: 18.1646
Lote  4/15 | Índices [ 300: 400] | Loss lote: 14.8295
Lote  5/15 | Índices [ 400: 500] | Loss lote: 12.0606
Lote  6/15 | Índices [ 500: 600] | Loss lote: 12.0543
Lote  7/15 | Índices [ 600: 700] | Loss lote: 8.8474
Lote  8/15 | Índices [ 700: 800] | Loss lote: 7.8111
Lote  9/15 | Índices [ 800: 900] | Loss lote: 7.0451
Lote 10/15 | Índices [ 900:1000] | Loss lote: 6.3738
Lote 11/15 | Índices [1000:1100] | Loss lote: 7.1667
Lote 12/15 | Índices [1100:1200] | Loss lote: 16.6758
Lote 13/15 | Índices [1200:1300] | Loss lote: 11.3502
Lote 14/15 | Índices [1300:1400] | Loss lote: 11.7678
Lote 15/15 | Índices [1400:1500] | Loss lote: 13.1425


En el primer lote, se tuvo un error de 30.917, mientras que en el ultimo lote se tuvo un error de 13.1425. Hay que mencionar qu edel lote q hasta el 10 la funcion de costos disminuyo, pero en los lostes de 12-15 aumento.